# Python Pokedex Project

## Step 1. Getting to know the project and dataset

Pokemon is a cult classic video game that ask of you to "catch 'em all" in order to complete the pokedex. The pokedex is a database that holds all relevant information once about a pokemon once they are obtained.

The objective of this project is to use Python's native list, dictionaries, tuples, and sets to organize and filter out meaningful data from the pokedex.

Each pokemon has a different name, size, ability, typing, ect. In order to work with the pokedex, we have its information in a csv file. 

Upon start up, there is a menu with options where the user can feed information in order to obtain specific information in a human-readable format.

## Step 2. Import modules

In [ ]:
import csv
import copy

TYPING = ['filler', 'bug', 'dark', 'dragon', 'electric', 'fairy', 'fight', 'fire', 'flying', 'ghost', 'grass', 'ground', 'ice', 'normal', 'poison', 'psychic', 'rock', 'steel', 'water']
EFFECTIVENESS = {0.25: "super effective", 0.5: "effective", 1.0: "normal", 2.0:"weak", 4.0:"super weak", 0:"resistant"}
MATCHUP_TYPES = {"resistant", "super effective", "effective", "normal",
                 "weak", "super weak"}
PROMPT = '''
\nTo make a selection, please enter an option 1-3:\n
\tOPTION 1: Find Pokemon
\tOPTION 2: Find Pokemon From Abilities
\tOPTION 3: Find Matchups\n'''

## Step 3. Create Functions to filter data

### effectiveness 
Parameters: column, case_effectiveness, return: tuple

In [ ]:
def effectiveness(column, case_effectiveness):

    case_effectiveness = float(case_effectiveness)  # Make all values floats
    
    effect = EFFECTIVENESS[case_effectiveness]  # return str name IE "weak"
    typing = TYPING[column]                     # Return type name IE "dark"
    return effect, typing

### open_file
Parameters: none, return: file pointer 

In [ ]:
def open_file():
    
    # Collect user input for filename
    file_name = input("Please enter a pokemon filename: ")
    
    while(True):                    # Repeat forever
        # Check if the file_name can be opened, otherwise reset loop
        try:                        # Attempt to open file
            file_pointer = open(file_name, encoding="utf-8")
            break                   # Break out of infinite loop
        except FileNotFoundError:   # If attempt failed
            print("This pokemon file does not exist. Please try again.")
            file_name = input("Please enter a pokemon filename: ")
            continue                # Reset the infinite loop
        
    return(file_pointer)

### read_file
Parameters: fp, return: data_map (dictionary)

In [ ]:
def read_file(fp):

    reader = csv.reader(fp)     # Read the excel sheet
    next(reader,None)           # Skip header

    data_map = {}               # Establish empty dictionary

    for line in reader:         # Main loop (iterates through each excel row)
        
        generation = line[39]
        name = line[30]
        hp = int(line[28])
        capture_rate = int(line[23])
        base_weight = float(line[38])
        base_speed = int(line[35])
        legendary = bool(int(line[40]))
        
        # Create empty set and insert pokemon ablility into it
        ability_set = set()                 # Establish empty set
        # Formatting cell information into workable information
        ability = line[0].strip('[\'')      # Strip first part of string
        ability = ability.strip(']\'')      # Strip last part of string
        ability = ability.replace("\', \'", ',')    # Replace middle with comma                                      
        ability = ability.split(',')        # Split and turn str into list      
        ability_set = set(ability)     
        
        # If there is no secondary type, it will be none inside the tuple
        if line[37] == "":
            line[37] = None
        types = (line[36], line[37])    # Creating tuple that is types
        
        # Establish new / nested dictionary variables if it does not exist yet
        if generation not in data_map:
            data_map[generation] = {}               # Name of generation
        if types not in data_map[generation]:
            data_map[generation][types] = {}        # Name of types 
        if name not in data_map[generation][types]:
            data_map[generation][types][name] = []  # Name of pokemon list

        # Establish a dictionary of effectiveness and sets for each
        effective_dict = {}
        super_effective_set = set()
        effective_set = set()
        normal_set = set()
        weak_set = set()
        super_weak_set = set()
        resistant_set = set()
        
        # Loop through each column of typings in the respective row
        for column in range(19):
            if column == 0:                         # Skip pokemon ability
                continue
            
            # Assign value to current column
            case_effectiveness = line[column]       # Represented by a int
            # Call on function effectiveness to return tuple IE (weak, dark)
            effect = effectiveness(column, case_effectiveness)
            
            # Add the typing name into it's respective set
            # Add to empty dictionary, IE {"effective": effective_set}
            if effect[0] == 'super effective':
                super_effective_set.add(effect[1])  
                effective_dict[effect[0]] = super_effective_set
            
            elif effect[0] == 'effective':
                effective_set.add(effect[1])
                effective_dict[effect[0]] = effective_set
                    
            elif effect[0] == 'normal':
                normal_set.add(effect[1])
                effective_dict[effect[0]] = normal_set
                    
            elif effect[0] == 'weak':
                weak_set.add(effect[1])
                effective_dict[effect[0]] = weak_set
                
            elif effect[0] == 'super weak':
                super_weak_set.add(effect[1])
                effective_dict[effect[0]] = super_weak_set
                
            elif effect[0] == 'resistant':
                resistant_set.add(effect[1])
                effective_dict[effect[0]] = resistant_set
        
        # Appending typing effectiveness into pokemon_list
        data_map[generation][types][name].append(effective_dict)
            
        # Appending all none elemental interactions into pokemon list
        data_map[generation][types][name].append(ability_set)
        data_map[generation][types][name].append(hp)
        data_map[generation][types][name].append(capture_rate)
        data_map[generation][types][name].append(base_weight)
        data_map[generation][types][name].append(base_speed)
        data_map[generation][types][name].append(legendary)
    
    return(data_map)

### find_pokemon
Parameters: nested dictionary, names (set of formatted user inputed names), return: dictionary

In [ ]:
def find_pokemon(pokedex, names):
    
    temp_dict = {}                          # Initialize empty dictionary
    poke = copy.deepcopy(pokedex)           # SDFISDF
    
    for name in names:                      # Iterate through set of names
        for key, value in poke.items():     # Iterate through generations
            # Iterate through typing in generations
            for key1, value1 in value.items():
                # if user_input names inside the names in nested dictionary
                if name in value1:
                    # Initialize a new list every loop
                    info = (value1[name])   # Pokemon info
                    del info[0]             # Delete Effectivenesses
                    info.append(key)        # Append generation
                    info.append(key1)       # Append typing
            
                    temp_dict[name] = info  # Put into dictionary -
                                            # (key = pokemon name)
    return (temp_dict)

### display_pokemon
Parameters: name, info (dictionary key and value), return: string

In [ ]:
def display_pokemon(name, info):   

    # Getting typing for display
    type_list = []                  # Initialize empty list
    for typ in info[7]:             # Iterate through typing, ie ('fire', None)
        if typ:                     # If it exist or =/ to None
            type_list.append(typ)   # Add to list, results in EX. [water] 
    # This section is used to turn list into a string
    if len(type_list) == 2:         # If there are two types
        typ = str(type_list[0]) + ', ' + str(type_list[1])
    else:                           # If there is one type
        typ = str(type_list[0])
        
    # Getting ability for display
    ability_list = []               # Initialize list
    for ab in info[0]:              # Iterate through ability
        ability_list.append(ab)     # Add to list
    # Organize abilities alphabetically 
    ability_list = sorted(ability_list)
    
    # This section is to turn ability list into a string
    ability = ''                    # Initialize empty string
    for i in ability_list:          # Iterate through ability list
        temp = str(i) + ', '        # Turn each list item into a formatted str
        ability += temp             # Add the formatted str to the empty string
    # Remove the last ', ' from the string
    ability = ability[0:-2]

    # Getting legendary status    
    if info[5] == True:
        legendary_status = "\tLegendary" 
    elif info[5] == False:
        legendary_status = "\tNot Legendary"  

    # Getting all the other values
    gen = str(info[6])
    hp = str(info[1])
    capture_rate = str(info[2])
    weight = str(info[3])
    speed = str(info[4])
    
    # The actual string that will be returned
    string = "\n" + name + "\n\tGen: " + gen + "\n\tTypes: " + typ
    string += "\n\tAbilities: " + ability + "\n\tHP: " + hp 
    string += "\n\tCapture Rate: " 
    string += capture_rate + "\n\tWeight: " + weight + "\n\tSpeed: " + speed
    string += '\n' + legendary_status
    
    return (string)

### find_pokemon_abilities
Parameters: nested dictionary, abilities (user input), return: set

In [ ]:
def find_pokemon_from_abilities(pokedex, abilities):
    
    # Initialize values
    temp_set = set()
    poke = copy.deepcopy(pokedex)
        
    # Iterate through generations
    for key, value in poke.items():
        
        # Iterate through typing in generations
        for key1, value1 in value.items():
            
            # Iterate through pokemon in typings
            # value2 = pokemon informations
            for key2, value2 in value1.items():
                    
                # If abilities in pokemon information list
                if abilities < value2[1]:
                    
                    # Add pokemon name into a set 
                    temp_set.add(key2)
                
    return(temp_set)        

### find_matchup
Parameters: nested dictionary, name, matchup_type, return: list of tuples

In [ ]:
def find_matchups(pokedex, name, matchup_type):
    
    # Initialize values
    temp_list = []
    poke = copy.deepcopy(pokedex)
    name_list = []
    effect_set = None   # What match_up meets user input IE rock, ice. Or None.
    
    # This for loop goes through pokemon and matches name to recieve a 
    # specific pokmone's typing
    for key, value in poke.items():             # Iterate through generation
        for key1, value1 in value.items():      # Iterate through typing
            for key2, value2 in value1.items(): # Iterate through name (Key2)
                
                # Append all pokemon names into name_list
                name_list.append(key2)
                if name == key2:                # If inputed name = dict name
                    # Getting type from dictionary value based on input type
                    effect_set = value2[0].get(matchup_type)
    
    # ALL ERRORS
    if matchup_type not in MATCHUP_TYPES:       # matchup_type is invalid
        return(None)
    if name not in name_list:                   # Pokemon name does not exist
        return(None)
    if effect_set == None:                      # No match_up types exist
        return(None)
                    
    # Collect name and type of pokemon 
    for key, value in poke.items():             # Iterate through generation
        for key1, value1 in value.items():      # Iterate through typing
            for key2, value2 in value1.items(): # Iterate through name (Key2)
                
                pokemon_type = set(key1) # Type into a set, IE {'ice', 'Rock'}
                # Subset, check if pokemon_type contains effect_set, IE Rock
                if effect_set < pokemon_type:

                    # Make a tuple of pokemon name and type
                    temp_tup = (key2, key1)
                    temp_list.append(temp_tup)
                    
    return temp_list

## Step 4. Create function to run program

In [ ]:
def main():
    print("Welcome to your personal Pokedex!\n")
    fp = open_file()               # Call for
    pokedex = read_file(fp)
    
    # Outer loop to quit out of everything when input is 'q'
    while(True):
        
        # Inner loop to repeat prompt and input until acceptable input chosen
        while(True):
            print(PROMPT)
            ACCEPTABLE = ('1', '2', '3', '4', 'q', 'Q')
            user_input = input("Enter an option: ")
            if user_input in ACCEPTABLE:
                break
            else:
                print('Invalid option {}'.format(user_input))
                continue
        
        # When user_input = 'q' break out of outer loop
        if user_input == 'q' or user_input == 'Q':
            break
                
        # Option 1 Find a pokemon
        if user_input == '1':
            # Get user input for pokemon name
            pokemon_name = input('\nEnter a list of pokemon names, ' +
                                 'separated by commas: ')
            
            # Remove all leading and ending space and split by commas
            pokemon_name = pokemon_name.replace(' ', '').split(',')
            temp = []
            for name in pokemon_name:   # Turn every element in list to title
                titled = name.title()
                temp.append(titled)
                
            # Turn list into a set
            pokemon_name_set = set(temp)
            
            # Call functions to display stuff
            information = find_pokemon(pokedex, pokemon_name_set)

            # Sorted to display pokemon alphabetically
            for key, value in sorted(information.items()):

                string = display_pokemon(key, value)
                print(string)
            
        
        # Option 2 Find pokemon from ability
        if user_input == '2':
            # Get user input for pokemon ability
            pokemon_ability = input("Enter a list of abilities, " +
                                    "separated by commas: ")
            
            # Remove all leading and ending space and split by commas
            pokemon_ability = pokemon_ability.split(',')
            temp = []
            for ability in pokemon_ability:   
                titled = ability.strip()
                titled = titled.title()
                temp.append(titled)       
                
            # Turn list into a set
            abilities = set(temp)
            
            # Call function for pokemon ability
            information = find_pokemon_from_abilities(pokedex, abilities)
            
            # Convert set into list and sort
            temp_list = list(information)
            temp_list = sorted(temp_list)
            print('Pokemon: ', end = ' ')
            
            # Print out each pokemon name and ending with ', '
            # Except for last pokemon which is just the name alone
            for pokemon in temp_list[:-1]:
                print('{},'.format(pokemon), end = ' ')
            print('{}'.format(temp_list[-1]))


        # Option 3 Find matchups
        if user_input == '3':
            # Get user input for pokemon name and match type
            pokemon_name = input('Enter a pokemon name: ')
            pokemon_match = input('Enter a matchup type: ')
            
            pokemon_name = pokemon_name.title()
            pokemon_match = pokemon_match.lower()
            
            match_list = find_matchups(pokedex, pokemon_name, pokemon_match)
            
            if match_list == None:
                print('Invalid input')
            
            if match_list != None:
                match_list = sorted(match_list)
            
                for i in match_list:
                    if i[1][1] == None:
                        print('{}: {}'.format(i[0], i[1][0]))
                    else:
                        print("{}: {}, {}".format(i[0], i[1][0], i[1][1]))

#### Conditional block

In [19]:
if __name__ == "__main__":
    main()

Welcome to your personal Pokedex!

Please enter a pokemon filename: pokemon.csv


To make a selection, please enter an option 1-3:

	OPTION 1: Find Pokemon
	OPTION 2: Find Pokemon From Abilities
	OPTION 3: Find Matchups

Enter an option: 1

Enter a list of pokemon names, separated by commas: Squirtle

Squirtle
	Gen: 1
	Types: water
	Abilities: Rain Dish, Torrent
	HP: 44
	Capture Rate: 45
	Weight: 9.0
	Speed: 43
	Not Legendary


To make a selection, please enter an option 1-3:

	OPTION 1: Find Pokemon
	OPTION 2: Find Pokemon From Abilities
	OPTION 3: Find Matchups

Enter an option: 1

Enter a list of pokemon names, separated by commas: Kyogre, Pikachu,Squirtle,     Turtwig

Kyogre
	Gen: 3
	Types: water
	Abilities: Drizzle
	HP: 100
	Capture Rate: 3
	Weight: 352.0
	Speed: 90
	Legendary

Pikachu
	Gen: 1
	Types: electric
	Abilities: Lightningrod, Static
	HP: 35
	Capture Rate: 190
	Weight: 6.0
	Speed: 90
	Not Legendary

Squirtle
	Gen: 1
	Types: water
	Abilities: Rain Dish, Torrent
	HP: 44
	C